# VERSION 2.0

In [90]:
%matplotlib qt

In [91]:
# importing required libraries
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint as pp
from sklearn.model_selection import train_test_split
from pprint import pprint
from sklearn.linear_model import LinearRegression
from texttable import Texttable
import math
from sklearn.metrics import r2_score

# getting the battery data
bs_all = [
    'B0005', 'B0006', 'B0007', 'B0018', 'B0025', 'B0026', 'B0027', 'B0028', 'B0029', 'B0030', 'B0031', 'B0032', 
    'B0033', 'B0034', 'B0036', 'B0038', 'B0039', 'B0040', 'B0041', 'B0042', 'B0043', 'B0044', 'B0045', 'B0046', 
    'B0047', 'B0048'
]
ds = {}
for b in bs_all:
    ds[b] = loadmat(f'DATA/{b}.mat')
    
types = {}
times = {}
ambient_temperatures = {}
datas = {}

for b in bs_all:
    x = ds[b][b]["cycle"][0][0][0]
    ambient_temperatures[b] = x['ambient_temperature']
    types[b] = x['type']
    times[b] = x['time']
    datas[b] = x['data']

# clubbing all the compatible batteries together
# Batteries are compatible if they were recorded under similar conditions
# And their data size match up
bs_compt = {}

for b in bs_all:
    sz = 0
    for j in range(datas[b].size):
        if types[b][j] == 'discharge':
            sz += 1
    if bs_compt.get(sz):
        bs_compt[sz].append(b)
    else: 
        bs_compt[sz] = [ b ]
pp(bs_compt)

{28: ['B0025', 'B0026', 'B0027', 'B0028'],
 40: ['B0029', 'B0030', 'B0031', 'B0032'],
 47: ['B0038', 'B0039', 'B0040'],
 67: ['B0041'],
 72: ['B0045', 'B0046', 'B0047', 'B0048'],
 112: ['B0042', 'B0043', 'B0044'],
 132: ['B0018'],
 168: ['B0005', 'B0006', 'B0007'],
 197: ['B0033', 'B0034', 'B0036']}


In [92]:
## CRITICAL TIME POINTS FOR A CYCLE
## We will only these critical points for furthur training

## TEMPERATURE_MEASURED
## => Time at highest temperature

## VOLTAGE_MEASURED
## => Time at lowest Voltage

## VOLTAGE_LOAD
## => First time it drops below 1 volt after 1500 time

def getTemperatureMeasuredCritical(tm, time):
    high = 0
    critical = 0
    for i in range(len(tm)):
        if (tm[i] > high):
            high = tm[i]
            critical = time[i]
    return critical

def getVoltageMeasuredCritical(vm, time):
    low = 1e9
    critical = 0
    for i in range(len(vm)):
        if (vm[i] < low):
            low = vm[i]
            critical = time[i]
    return critical

def getVoltageLoadCritical(vl, time):
    for i in range(len(vl)):
        if (time[i] > 1500 and vl[i] < 1):
            return time[i]
    return -1

## Considering All the Compatible Batteries Together

In [95]:
## Data Structure
    # Cycles[battery][param][cycle]
    # Cycles[battery][Capacity][cycle]

Cycles = {}
params = ['Temperature_measured', 'Voltage_measured', 'Voltage_load', 'Time']

results = Texttable()
results.add_row(['Compatible Batteries', 'Cycles', 'MAE', 'RMSE', 'R2 Score' ])

# iterate over all the battery sets
for bs_cmpt in bs_compt:
    # getting data for a given set
    bs = bs_compt[bs_cmpt]
    for b in bs:
        Cycles[b] = {}
        for param in params:
            Cycles[b][param] = []
            for j in range(datas[b].size):
                if types[b][j] == 'discharge':
                    Cycles[b][param].append(datas[b][j][param][0][0][0])

        cap = []
        for j in range(datas[b].size):
            if types[b][j] == 'discharge':
                cap.append(datas[b][j]['Capacity'][0][0][0][0])
        Cycles[b]['Capacity'] = np.array(cap)
        Cycles[b]['count'] = len(Cycles[b][params[0]])
        
    # preparing data for regression model
    temperature_measured = []
    voltage_measured = []
    voltage_load = []
    capacity = []

    for b in bs:        
        for c in Cycles[b]['Capacity']:
            capacity.append(c)
        for i in range(Cycles[b]['count']):
            temperature_measured.append(getTemperatureMeasuredCritical(Cycles[b]['Temperature_measured'][i], Cycles[b]['Time'][i]))
            voltage_measured.append(getVoltageMeasuredCritical(Cycles[b]['Voltage_measured'][i], Cycles[b]['Time'][i]))
            voltage_load.append(getVoltageLoadCritical(Cycles[b]['Voltage_load'][i], Cycles[b]['Time'][i]))
    
    # creating the model
    X = []
    for i in range(len(temperature_measured)):
        X.append(np.array([temperature_measured[i], voltage_measured[i], voltage_load[i]]))
    X = np.array(X)
    y = np.array(capacity)

    # creating train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

    # fitting the model
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    
    # test
    y_pred = regressor.predict(X_test)
    
    # model evaluation
    diff = 0
    total = 0
    rmse = 0
    for i in range(len(y_test)):
        diff += abs(y_test[i] - y_pred[i])
        rmse += ((y_test[i] - y_pred[i]) * (y_test[i] - y_pred[i]))
        total += y_test[i]
    diff /= len(y_test)
    total /= len(y_test)
    rmse = math.sqrt(rmse / len(y_test))
#     accuracy = ((total - diff) / total) * 100
    accuracy = r2_score(y_test, y_pred)
    
    # Adding evaluation to result array to print in a table
    results.add_row([ str(bs), str(Cycles[bs[0]]['count']), diff, rmse, accuracy ])
    
# printing results
print(f'Evaluation: Clubbing Compatible Batteries\n{results.draw()}')

Evaluation: Clubbing Compatible Batteries
+--------------------------------------+--------+-------+-------+----------+
| Compatible Batteries                 | Cycles | MAE   | RMSE  | R2 Score |
+--------------------------------------+--------+-------+-------+----------+
| ['B0005', 'B0006', 'B0007']          | 168    | 0.013 | 0.015 | 0.995    |
+--------------------------------------+--------+-------+-------+----------+
| ['B0018']                            | 132    | 0.002 | 0.003 | 1.000    |
+--------------------------------------+--------+-------+-------+----------+
| ['B0025', 'B0026', 'B0027', 'B0028'] | 28     | 0.005 | 0.008 | 0.990    |
+--------------------------------------+--------+-------+-------+----------+
| ['B0029', 'B0030', 'B0031', 'B0032'] | 40     | 0.008 | 0.009 | 0.982    |
+--------------------------------------+--------+-------+-------+----------+
| ['B0033', 'B0034', 'B0036']          | 197    | 0.033 | 0.060 | 0.939    |
+---------------------------------

## Considering One Battery at a time

In [97]:
## Data Structure
    # Cycles[battery][param][cycle]
    # Cycles[battery][Capacity][cycle]

Cycles = {}
params = ['Temperature_measured', 'Voltage_measured', 'Voltage_load', 'Time']

results = Texttable()
results.add_row(['Battery', 'Cycles', 'MAE', 'RMSE', 'R2 Score' ])

# iterate over all the battery sets
for b in bs_all:
    # getting data for a given set
    bs = [ b ]
    for b in bs:
        Cycles[b] = {}
        for param in params:
            Cycles[b][param] = []
            for j in range(datas[b].size):
                if types[b][j] == 'discharge':
                    Cycles[b][param].append(datas[b][j][param][0][0][0])

        cap = []
        for j in range(datas[b].size):
            if types[b][j] == 'discharge':
                cap.append(datas[b][j]['Capacity'][0][0][0][0])
        Cycles[b]['Capacity'] = np.array(cap)
        Cycles[b]['count'] = len(Cycles[b][params[0]])
        
    # preparing data for regression model
    temperature_measured = []
    voltage_measured = []
    voltage_load = []
    capacity = []

    for b in bs:        
        for c in Cycles[b]['Capacity']:
            capacity.append(c)
        for i in range(Cycles[b]['count']):
            temperature_measured.append(getTemperatureMeasuredCritical(Cycles[b]['Temperature_measured'][i], Cycles[b]['Time'][i]))
            voltage_measured.append(getVoltageMeasuredCritical(Cycles[b]['Voltage_measured'][i], Cycles[b]['Time'][i]))
            voltage_load.append(getVoltageLoadCritical(Cycles[b]['Voltage_load'][i], Cycles[b]['Time'][i]))
    
    # creating the model
    X = []
    for i in range(len(temperature_measured)):
        X.append(np.array([temperature_measured[i], voltage_measured[i], voltage_load[i]]))
    X = np.array(X)
    y = np.array(capacity)

    # creating train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

    # fitting the model
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    
    # test
    y_pred = regressor.predict(X_test)
    
    # model evaluation
    diff = 0
    total = 0
    rmse = 0
    for i in range(len(y_test)):
        diff += abs(y_test[i] - y_pred[i])
        rmse += ((y_test[i] - y_pred[i]) * (y_test[i] - y_pred[i]))
        total += y_test[i]
    diff /= len(y_test)
    total /= len(y_test)
    rmse = math.sqrt(rmse / len(y_test))
#     accuracy = ((total - diff) / total) * 100
    accuracy = r2_score(y_test, y_pred)
    
    # Adding evaluation to result array to print in a table
    results.add_row([ bs[0], str(Cycles[bs[0]]['count']), diff, rmse, accuracy ])
    
# printing results
print(f'Evaluation: Considering Individual Battery\n{results.draw()}')

Evaluation: Considering Individual Battery
+---------+--------+-------+-------+----------+
| Battery | Cycles | MAE   | RMSE  | R2 Score |
+---------+--------+-------+-------+----------+
| B0005   | 168    | 0.000 | 0.000 | 1.000    |
+---------+--------+-------+-------+----------+
| B0006   | 168    | 0.002 | 0.003 | 1.000    |
+---------+--------+-------+-------+----------+
| B0007   | 168    | 0.002 | 0.003 | 1.000    |
+---------+--------+-------+-------+----------+
| B0018   | 132    | 0.002 | 0.003 | 1.000    |
+---------+--------+-------+-------+----------+
| B0025   | 28     | 0.003 | 0.003 | 0.952    |
+---------+--------+-------+-------+----------+
| B0026   | 28     | 0.008 | 0.011 | 0.994    |
+---------+--------+-------+-------+----------+
| B0027   | 28     | 0.003 | 0.005 | 0.787    |
+---------+--------+-------+-------+----------+
| B0028   | 28     | 0.000 | 0.001 | 0.999    |
+---------+--------+-------+-------+----------+
| B0029   | 40     | 0.005 | 0.006 | 0.978   